# 데이터 불러오기

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = (SparkConf().setMaster("k8s://https://ip_address:port") # Your master address name
        .set("spark.kubernetes.container.image", "joron1827/pyspark:latest") # Spark image name
        .set("spark.driver.port", "2222") # Needs to match svc
        .set("spark.driver.blockManager.port", "7777")
        .set("spark.driver.host", "driver-service.jupyterhub.svc.cluster.local") # Needs to match svc
        .set("spark.driver.bindAddress", "0.0.0.0")
        .set("spark.kubernetes.namespace", "spark")
        .set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
        .set("spark.kubernetes.authenticate.serviceAccountName", "spark")
        .set("spark.executor.instances", "2")
        .set("spark.kubernetes.container.image.pullPolicy", "IfNotPresent")
        .set("spark.app.name", "tutorial_app")
        .set("spark.executor.memory", "4g"))

In [ ]:
# create SparkSession
spark = SparkSession.builder.config(conf=conf).getOrCreate()

/home/jovyan/.local/lib/python3.9/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


23/05/02 11:47:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 하둡으로부터 데이터 불러오기

In [ ]:
df = spark.read.parquet("hdfs://ip_address:port/recipe/table/*.parquet")
df.show()

+----------------+----------+----------------------------------+-----------+--------+------------------------------+---------------------------------+-----------+-----------+-----------+
|          author|createDate|                              name|ratingValue|recipeId|              recipeIngredient|               recipeInstructions|recipeYield|reviewCount|  totalTime|
+----------------+----------+----------------------------------+-----------+--------+------------------------------+---------------------------------+-----------+-----------+-----------+
|        아랄랄라|2017-12-19|  (집에서 즐기는 술안주) 동파육...|          5| 6881118|   [통 삼겹 1kg, 청경채 3개...|  [홈 파티 요리  고급스러운 통...| 3 servings|          5|PT30H-1710M|
|        강철새잎|2019-02-20|  #잡채만들기 #간단하게 만드는 ...|          0| 6906965|    [당근 1/2개, 양파 1개, ...| [일단 잡채에 들어가는 모든 재...| 6 servings|          0|      PT30M|
|        춤추는곰|2019-03-21|  푹 익힌 닭에 깊은 양념맛이 좋...|          5| 6908917| [닭볶음용 닭 1마리, 양파 1...|[요즘 마트에가면 닭볶음용 온가...| 3 servings|         

## Spark 데이터프레임을 Pandas 데이터프레임으로 변환

In [ ]:
df = df.toPandas()
df.head()

,author,createDate,name,ratingValue,recipeId,recipeIngredient,recipeInstructions,recipeYield,reviewCount,totalTime
0,아랄랄라,2017-12-19,(집에서 즐기는 술안주) 동파육 - 통삼겹 요리 / 홈파티 요리,5,6881118,"[통 삼겹 1kg, 청경채 3개, 대파 1대, 양파 1개, 통마늘 8개, 생강 1/...",[홈 파티 요리 고급스러운 통삼겹 요리 동파육 만들기 ♬♪\n※ 동파육 잡내 제거...,3 servings,5,PT30H-1710M
1,강철새잎,2019-02-20,#잡채만들기 #간단하게 만드는 잡채 #한꺼번에 볶아내면 금방 뚝딱 만드는 잡채!!!,0,6906965,"[당근 1/2개, 양파 1개, 표고버섯 4개, 목이버섯 1컵, 어묵 2장, 호박고지...","[일단 잡채에 들어가는 모든 재료들은 무조건 채를 썰어서 준비합니다, 건표고버섯채...",6 servings,0,PT30M
2,춤추는곰,2019-03-21,푹 익힌 닭에 깊은 양념맛이 좋은 닭볶음탕,5,6908917,"[닭볶음용 닭 1마리, 양파 1/2개, 당근 1/3개, 감자 1개, 대파 1줌, 고...",[요즘 마트에가면 닭볶음용 온가족세트 닭이 있더라구요~ 할인도 자주해서 종종 사오는...,3 servings,7,PT30M
3,왕눈이2,2019-09-15,맵지않아 더욱더 맛나다는 백김치의 비법은?,5,6919155,"[배추 2포기, 양파 1/2개, 사과 1/2개, 배 중간크기1개, 당근 1/2개, ...",[배추 2포기를 4등분으로 잘라주신후 큰볼에 물을 담아 소금 1컵을 풀어서 자른배추...,6 servings,3,PT2H
4,EMYY,2019-07-01,메밀부꾸미,0,6915060,"[메밀가루 3T, 계란 1개, 우유 3T, 3층 치즈 2장, 아몬드슬라이스 조금, ...","[껍질벗긴 메밀3T를 분쇄기로 갈아요, (시판 메밀가루 3T 쓰셔도 되구요, )\...",1 servings,0,PT15M


In [ ]:
df.to_csv("recipe.csv", index=False)

In [ ]:
spark.stop()

23/05/02 12:32:38 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.


# 데이터 전처리

In [ ]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv("recipe.csv")
df["recipeIngredient"] = df["recipeIngredient"].apply(literal_eval)
df.head()

,author,createDate,name,ratingValue,recipeId,recipeIngredient,recipeInstructions,recipeYield,reviewCount,totalTime
0,아랄랄라,2017-12-19,(집에서 즐기는 술안주) 동파육 - 통삼겹 요리 / 홈파티 요리,5,6881118,"[통 삼겹 1kg, 청경채 3개, 대파 1대, 양파 1개, 통마늘 8개, 생강 1/...",['홈 파티 요리 고급스러운 통삼겹 요리 동파육 만들기 ♬♪\n※ 동파육 잡내 제...,3 servings,5,PT30H-1710M
1,강철새잎,2019-02-20,#잡채만들기 #간단하게 만드는 잡채 #한꺼번에 볶아내면 금방 뚝딱 만드는 잡채!!!,0,6906965,"[당근 1/2개, 양파 1개, 표고버섯 4개, 목이버섯 1컵, 어묵 2장, 호박고지...","['일단 잡채에 들어가는 모든 재료들은 무조건 채를 썰어서 준비합니다', ' 건표고...",6 servings,0,PT30M
2,춤추는곰,2019-03-21,푹 익힌 닭에 깊은 양념맛이 좋은 닭볶음탕,5,6908917,"[닭볶음용 닭 1마리, 양파 1/2개, 당근 1/3개, 감자 1개, 대파 1줌, 고...",['요즘 마트에가면 닭볶음용 온가족세트 닭이 있더라구요~ 할인도 자주해서 종종 사오...,3 servings,7,PT30M
3,왕눈이2,2019-09-15,맵지않아 더욱더 맛나다는 백김치의 비법은?,5,6919155,"[배추 2포기, 양파 1/2개, 사과 1/2개, 배 중간크기1개, 당근 1/2개, ...",['배추 2포기를 4등분으로 잘라주신후 큰볼에 물을 담아 소금 1컵을 풀어서 자른배...,6 servings,3,PT2H
4,EMYY,2019-07-01,메밀부꾸미,0,6915060,"[메밀가루 3T, 계란 1개, 우유 3T, 3층 치즈 2장, 아몬드슬라이스 조금, ...","['껍질벗긴 메밀3T를 분쇄기로 갈아요', ' (시판 메밀가루 3T 쓰셔도 되구요'...",1 servings,0,PT15M


## 식재료 맞춤법 교정 (네이버 맞춤법 검사기)

In [ ]:
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import aiohttp
import asyncio
import json

url = "https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy"
headers = {
    "Authority": 'm.search.naver.com',
    "Accept": '*/*',
    "Accept-Encoding": 'gzip, deflate, br',
    "Accept-Language": 'en-US,en;q=0.9,ko-KR;q=0.8,ko;q=0.7',
    "Sec-Ch-Ua": '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    "Sec-Ch-Ua-Mobile": '?0',
    "Sec-Ch-Ua-Platform": '"macOS"',
    "Sec-Fetch-Dest": 'script',
    "Sec-Fetch-Mode": 'no-cors',
    "Sec-Fetch-Site": 'same-site',
    "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}

async def fetch(text: str, session: aiohttp.ClientSession, semaphore: asyncio.Semaphore):
    params = {"_callback": "window.__jindo2_callback._spellingCheck_0", "q": text, "where":"nexearch", "color_blindness":0}
    try:
        async with semaphore:
            async with session.get(url, params=params, headers=headers) as response:
                data = (await response.text())[42:-2]
            await asyncio.sleep(1)
            result = json.loads(data)
            checked_word = result["message"]["result"]["html"]
            checked_word = BeautifulSoup(checked_word, "lxml").text
    except: checked_word = str()
    return checked_word

semaphore = asyncio.Semaphore(10)
session = aiohttp.ClientSession()
food = await tqdm.gather(*[fetch(', '.join(text), session, semaphore) for text in df["recipeIngredient"].tolist()])
await session.close()
df["x"] = food
df.head()

/home/jovyan/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 11/8208 [00:02<24:43,  5.53it/s] /usr/local/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|██████████| 8208/8208 [14:18<00:00,  9.56it/s]


,author,createDate,name,ratingValue,recipeId,recipeIngredient,recipeInstructions,recipeYield,reviewCount,totalTime,x
0,아랄랄라,2017-12-19,(집에서 즐기는 술안주) 동파육 - 통삼겹 요리 / 홈파티 요리,5,6881118,"[통 삼겹 1kg, 청경채 3개, 대파 1대, 양파 1개, 통마늘 8개, 생강 1/...",['홈 파티 요리 고급스러운 통삼겹 요리 동파육 만들기 ♬♪\n※ 동파육 잡내 제...,3 servings,5,PT30H-1710M,"통 삼겹 1kg, 청경채 3개, 대파 1대, 양파 1개, 통마늘 8개, 생강 1/2..."
1,강철새잎,2019-02-20,#잡채만들기 #간단하게 만드는 잡채 #한꺼번에 볶아내면 금방 뚝딱 만드는 잡채!!!,0,6906965,"[당근 1/2개, 양파 1개, 표고버섯 4개, 목이버섯 1컵, 어묵 2장, 호박고지...","['일단 잡채에 들어가는 모든 재료들은 무조건 채를 썰어서 준비합니다', ' 건표고...",6 servings,0,PT30M,"당근 1/2개, 양파 1개, 표고버섯 4개, 목이버섯 1컵, 어묵 2장, 호박고지 ..."
2,춤추는곰,2019-03-21,푹 익힌 닭에 깊은 양념맛이 좋은 닭볶음탕,5,6908917,"[닭볶음용 닭 1마리, 양파 1/2개, 당근 1/3개, 감자 1개, 대파 1줌, 고...",['요즘 마트에가면 닭볶음용 온가족세트 닭이 있더라구요~ 할인도 자주해서 종종 사오...,3 servings,7,PT30M,"닭볶음용 닭 1마리, 양파 1/2개, 당근 1/3개, 감자 1개, 대파 1줌, 고추..."
3,왕눈이2,2019-09-15,맵지않아 더욱더 맛나다는 백김치의 비법은?,5,6919155,"[배추 2포기, 양파 1/2개, 사과 1/2개, 배 중간크기1개, 당근 1/2개, ...",['배추 2포기를 4등분으로 잘라주신후 큰볼에 물을 담아 소금 1컵을 풀어서 자른배...,6 servings,3,PT2H,"배추 2포기, 양파 1/2개, 사과 1/2개, 배 중간 크기 1개, 당근 1/2개,..."
4,EMYY,2019-07-01,메밀부꾸미,0,6915060,"[메밀가루 3T, 계란 1개, 우유 3T, 3층 치즈 2장, 아몬드슬라이스 조금, ...","['껍질벗긴 메밀3T를 분쇄기로 갈아요', ' (시판 메밀가루 3T 쓰셔도 되구요'...",1 servings,0,PT15M,"메밀가루 3T, 계란 1개, 우유 3T, 3층 치즈 2장, 아몬드 슬라이스 조금, ..."


In [ ]:
df.to_csv("recipe2.csv", index=False)

## 식재료로부터 단위를 제거

In [ ]:
import re

def unique(*elements, empty=False, **kwargs):
    array = list()
    for element in elements:
        if element not in array: array.append(element)
    return array

exclude = ["약간씩", "중간 크기", "아주 조금", "조금", "적당량", "넉넉한", "적당히", "약간", "크게", "듬뿍", "제일 작은 거", "준비재료:", "절일 때", "잘게 썬", "조금 큰", "넉넉히", "취향껏", "약간", "절임용", "슈거나", "손질 냉동", "원하는 만큼", "전량", "얇은", "마트에서 파는", "보이진 않지만", "송송 썬", "큰 것", "톡톡", "드실 만큼", "썰어서", "불린 것", "먹을 만큼", "약간씩", "​", "있으면", "크게", "각각", "고기 밑간 용", "소주잔", "소주컵", "다진 것", "세척 때", "데칠 때", "세척용", "한 묶음", "비비고", "다짐 육", "작은 것", "채 썬", "원하는 양", "잘게 썰어준", "넉넉", "흰 부분", "입맛에 따라", "큰 것", "머그컵", "시판 치킨", "간 것", "한 봉지", "* ", "큰 스푼", "굵은", "찐", "한 마리", "불고깃감", "작은 것", "말린", "삶은 것", "삼겹살 부분", " L", "냉장고 있는", "앞다리살덩어리", "봉긋하게", "초록색 부분", "흰 부분", "데친 것", "한 꼬집", "고봉으로", "개인 간에 맞게", "생략 가능", "송송 썬", "백숙용 영계", "엄지손가락만 한", "종이컵", "머그잔으로", "소량", "잠길 만큼", "시판", "마트에서 파는", "보이진 않지만", "작은 크기로", "밑간 한", "뼈 없는", "식성에 따라 가감", "바로 먹을 때", "푹 잠길 정도로", "듬뿍", "국내산", "고운", "조미 안된", "매운 거", "반 개", "빈 봉지", "빈캔", "삶은", "한 그릇", "➊", "삶아서 으깬", "큰 거", "큰놈", "중간 크기", "국그릇으로", "종이컵 기준", "주먹만 한 크기", "작은 사이즈", "손바닥만 한 크기", "산토리 품종 고기", "선택", "삶은 상태의", "남은 것", "본인이 먹을 만큼", "어슷 썬", "크게", "기호에 따라", ".", " 톡", "꼭 짠 것", "간 것", "손바닥 크기만큼", "많이", "주먹 크기 정도", "말린 것", "썬 것", "찌거나 구운 것", "필요에 따라", "큰 거", "적당히", "중간 크기", "굵은 걸로", "작은 거", "작은 것", "합해서", "한 개", "밥 수저 기준", "조금", "밥그릇", "채 썬", "작은 한 줌", "준비", "큰 술", "기호에 따라", "국거리용", "질지 않을 정도로", "주먹 크기", "입맛에 맞게", "반 숟가락", "말린 것", "한 그릇", "조금", "조금", "약간", "적당히", "살짝", "톡톡", "적정량", "적당량", "원하는만큼", "크게", "취향껏", "1봉지", "취향것", "종이컵", "듬뿍", "소주잔", "한 줌", "드실만큼", "1개", "1.5스푼", "小", "큰 스푼", "작은", "소량", "넉넉히", "큰 것", "많이", "소량", "크기", "선택", "한묶음", "반봉지", "잘라논거", "한봉지", "기호에 맞게", "큰한줌", "다진것", "썰은 거", "썰은것", "적당", "큰 거", "한개"]
exclude = unique(*exclude)
replace = {"파 프리카":"파프리카", "고정시킬 꽂이(이쑤시개)":"이쑤시개", "밀가루(부침가루/튀김가루/소금)":"밀가루", "편만 늘":"편마늘", "소면 줌":"소면", "빵(샌드위치 빵/파니니 빵/토르티야/햄버거 번/핫도그 번) 적당량":"빵", "로터스 또는 기타 스낵":"스낵", "섩탕 과일의":"과일", "전분 물 농도에 맞게":"전분", "칵테일새우(크기는 취향대로) 약":"칵테일새우", "귤 절여줄 설탕":"설탕", "통마늘 (다진 마늘":"통마늘", "땅콩(견과류) 적당량":"땅콩", " or ":" 또는 ", "or":" 또는 ", " / ":" 또는 ", "샐러리(":"샐러리", "귤+레몬":"귤, 레몬", "버터 한 조각(":"버터", "쌀 한 줌(쌀가루)":"쌀가루", "위에 뿌릴 굵게 다진 피스타치오 약간":"피스타치오", "찹쌀 풀 욕 물":"물", "닭볶음탕용으로 손질된 닭":"닭", "+ ":" 또는 ", "큰 술 - 밥숟가락 계량":"큰 술", "컵 - 종이컵 계량":"", "인원 -":"", "수비 드림 IQF 큐브 닭 가슴살":"닭가슴살", "하드 파마산 치즈 간 것":"치즈", "오뚝이 옛날 사골곰탕 국물":"사골곰탕 국물", "닭 가슴 살":"닭가슴살", "닭 안심 살":"닭안심살", "닭 다리 살":"닭다리살", "닭 다리":"닭다리", "닭 가슴":"닭가슴", "닭 안심":"닭안심", "전자레인지용 가라 알게 (코스트코)":"가라아게", "먹는 물 종이컵으로":"물", "황기 등 삼계탕 팩":"삼계탕 팩", "모둠해물(가리비&바지락&오징어&홍합&새우 살)":"모둠 해물", "계란 물 : 계란":"계랸", "적양배추 파프리카와 동일한 양":"적양배추", "파프리카 쩍 양배추와 동일한 양":"파프리카", "돼지고 기":"돼지고기", "쌈 무":"쌈무", "육사 시 미용 고기":"고기", "건곤 드레 나물":"곤드레나물", "경기미 참 드림 쌀":"쌀", "3인분 기준 스파게티 면 3인분":"스파게티 면", "다짐 육":"다짐육", "데칠 때 넣는 소금":"소금", "닭 날개":"닭날개", "2배 식초 1/2종이 컵":"식초", "고기가 없을 경우 베이컨":"고기", "곱게 갈아준 깨 크게":"깨", "원 레시피에는 돼지갈비":"돼지갈비", "돼지고기 A 지방 선택":"돼지고기", "돼지고기 삶을 때 사용할 된장":"된장", "11인치 타르트 팬 1개":"타르트 팬", "물 옥수수가 잠길 정도까지":"물", "육수에 넣었던 멸치와 다시마":"멸치와 다시마", ". ":", ", "디즈니 프린세스 후리가 케":"후리카케", "설탕/매실청/올리고당 등":"설탕", "백설 오천 년의 신비 명품 천일염":"천일염", "작은 송이로 손질한 브로콜리":"브로콜리", "아기 단계에 맞는 치즈":"치즈", "명절에 남은 과일과 야채들":"과일과 야채", "스파게티 동전 오백 원 굵기만큼":"스파게티", "백합 바지락 등의 조개":"조개", "통 삼겹살이 잠길 만큼의 물":"물", "식요융":"식용유"}
replace = dict(replace, **{text:"" for text in exclude})

def re_get(pattern: str, string: str, default=str(), groups=False, **kwargs) -> str:
    if not re.search(pattern, string): return default
    catch = re.search(pattern, string).groups()
    return catch[0] if catch and not groups else catch

def parse(food: str):
    if re.search("^\d", food): return (food, str())
    pattern = "(\d+.*)$"
    return (prep(re.sub(pattern, '', food).strip()), re_get(pattern, food).strip())

def prep(food: str):
    for org, to in replace.items():
        food = food.replace(org, to)
    food = re.sub("\([^)]*\)", '', food).strip()
    food = re.sub("\(.*$", '', food).strip()
    return food

In [ ]:
# 식재료명/단위 분리 여부 테스트

from itertools import chain

df["x"] = df["x"].apply(lambda x: x.split(', '))

data = list(chain.from_iterable(list(map(parse, x)) for x in df["x"]))
pd.DataFrame(data).to_csv("food.csv", index=False)
data[0]

('통 삼겹', '1kg')

In [ ]:
def parse(food: str):
    if re.search("^\d", food): return (food, str())
    pattern = "(\d+.*)$"
    return prep(re.sub(pattern, '', food).strip())

def map_food(x):
    data = [parse(food) for food in x]
    return [food for food in data if food]

df["x"] = df["x"].apply(map_food)
df.head() # 4번 인덱스 등 올바르게 처리되지 않는 부분에 대해 수작업으로 전처리

,author,createDate,name,ratingValue,recipeId,recipeIngredient,recipeInstructions,recipeYield,reviewCount,totalTime,x
0,아랄랄라,2017-12-19,(집에서 즐기는 술안주) 동파육 - 통삼겹 요리 / 홈파티 요리,5,6881118,"[통 삼겹 1kg, 청경채 3개, 대파 1대, 양파 1개, 통마늘 8개, 생강 1/...",['홈 파티 요리 고급스러운 통삼겹 요리 동파육 만들기 ♬♪\n※ 동파육 잡내 제...,3 servings,5,PT30H-1710M,"[통 삼겹, 청경채, 대파, 양파, 통마늘, 생강, 통후추, 월계수잎, 진간장, 굴..."
1,강철새잎,2019-02-20,#잡채만들기 #간단하게 만드는 잡채 #한꺼번에 볶아내면 금방 뚝딱 만드는 잡채!!!,0,6906965,"[당근 1/2개, 양파 1개, 표고버섯 4개, 목이버섯 1컵, 어묵 2장, 호박고지...","['일단 잡채에 들어가는 모든 재료들은 무조건 채를 썰어서 준비합니다', ' 건표고...",6 servings,0,PT30M,"[당근, 양파, 표고버섯, 목이버섯, 어묵, 호박고지, 돼지고기 잡채용, 시금치나물..."
2,춤추는곰,2019-03-21,푹 익힌 닭에 깊은 양념맛이 좋은 닭볶음탕,5,6908917,"[닭볶음용 닭 1마리, 양파 1/2개, 당근 1/3개, 감자 1개, 대파 1줌, 고...",['요즘 마트에가면 닭볶음용 온가족세트 닭이 있더라구요~ 할인도 자주해서 종종 사오...,3 servings,7,PT30M,"[닭볶음용 닭, 양파, 당근, 감자, 대파, 고추장, 다진 마늘, 맛술, 고춧가루,..."
3,왕눈이2,2019-09-15,맵지않아 더욱더 맛나다는 백김치의 비법은?,5,6919155,"[배추 2포기, 양파 1/2개, 사과 1/2개, 배 중간크기1개, 당근 1/2개, ...",['배추 2포기를 4등분으로 잘라주신후 큰볼에 물을 담아 소금 1컵을 풀어서 자른배...,6 servings,3,PT2H,"[배추, 양파, 사과, 배, 당근, 무, 쪽파, 홍고추, 골드 키위, 소금, 새우젓..."
4,EMYY,2019-07-01,메밀부꾸미,0,6915060,"[메밀가루 3T, 계란 1개, 우유 3T, 3층 치즈 2장, 아몬드슬라이스 조금, ...","['껍질벗긴 메밀3T를 분쇄기로 갈아요', ' (시판 메밀가루 3T 쓰셔도 되구요'...",1 servings,0,PT15M,"[메밀가루, 계란, 우유, (3층 치즈 2장, ), 아몬드 슬라이스, 건 크랜베리,..."


In [ ]:
df.to_csv("recipe3.csv", index=False)

In [ ]:
', '.join(df["x"][2218])

'밥, 식용유, 양파, 고추참치캔, 마늘, 파프리카, 다진 파, 참기름'

# 코사인 유사도

In [ ]:
pip install scikit-learn==0.24.2

In [10]:
data = df.copy()
data["recipeIngredient"] = data["recipeIngredient"].apply(lambda x: ','.join(x))
data.head()

,author,createDate,name,ratingValue,recipeId,recipeIngredient,recipeInstructions,recipeYield,reviewCount,totalTime
0,아랄랄라,2017-12-19,(집에서 즐기는 술안주) 동파육 - 통삼겹 요리 / 홈파티 요리,5,6881118,"통 삼겹 1kg,청경채 3개,대파 1대,양파 1개,통마늘 8개,생강 1/2개,통후추...",[홈 파티 요리 고급스러운 통삼겹 요리 동파육 만들기 ♬♪\n※ 동파육 잡내 제거...,3 servings,5,PT30H-1710M
1,강철새잎,2019-02-20,#잡채만들기 #간단하게 만드는 잡채 #한꺼번에 볶아내면 금방 뚝딱 만드는 잡채!!!,0,6906965,"당근 1/2개,양파 1개,표고버섯 4개,목이버섯 1컵,어묵 2장,호박고지 1/2컵,...","[일단 잡채에 들어가는 모든 재료들은 무조건 채를 썰어서 준비합니다, 건표고버섯채...",6 servings,0,PT30M
2,춤추는곰,2019-03-21,푹 익힌 닭에 깊은 양념맛이 좋은 닭볶음탕,5,6908917,"닭볶음용 닭 1마리,양파 1/2개,당근 1/3개,감자 1개,대파 1줌,고추장 3T,...",[요즘 마트에가면 닭볶음용 온가족세트 닭이 있더라구요~ 할인도 자주해서 종종 사오는...,3 servings,7,PT30M
3,왕눈이2,2019-09-15,맵지않아 더욱더 맛나다는 백김치의 비법은?,5,6919155,"배추 2포기,양파 1/2개,사과 1/2개,배 중간크기1개,당근 1/2개,무우 1/2...",[배추 2포기를 4등분으로 잘라주신후 큰볼에 물을 담아 소금 1컵을 풀어서 자른배추...,6 servings,3,PT2H
4,EMYY,2019-07-01,메밀부꾸미,0,6915060,"메밀가루 3T,계란 1개,우유 3T,3층 치즈 2장,아몬드슬라이스 조금,건크랜베리 ...","[껍질벗긴 메밀3T를 분쇄기로 갈아요, (시판 메밀가루 3T 쓰셔도 되구요, )\...",1 servings,0,PT15M


In [36]:
data["mergeIngredient"] = data.apply(lambda x: x["name"]+x["recipeIngredient"], axis=1)
data.head()

,author,createDate,name,ratingValue,recipeId,recipeIngredient,recipeInstructions,recipeYield,reviewCount,totalTime,mergeIngredient
0,아랄랄라,2017-12-19,(집에서 즐기는 술안주) 동파육 - 통삼겹 요리 / 홈파티 요리,5,6881118,"통 삼겹 1kg,청경채 3개,대파 1대,양파 1개,통마늘 8개,생강 1/2개,통후추...",[홈 파티 요리 고급스러운 통삼겹 요리 동파육 만들기 ♬♪\n※ 동파육 잡내 제거...,3 servings,5,PT30H-1710M,"(집에서 즐기는 술안주) 동파육 - 통삼겹 요리 / 홈파티 요리통 삼겹 1kg,청경..."
1,강철새잎,2019-02-20,#잡채만들기 #간단하게 만드는 잡채 #한꺼번에 볶아내면 금방 뚝딱 만드는 잡채!!!,0,6906965,"당근 1/2개,양파 1개,표고버섯 4개,목이버섯 1컵,어묵 2장,호박고지 1/2컵,...","[일단 잡채에 들어가는 모든 재료들은 무조건 채를 썰어서 준비합니다, 건표고버섯채...",6 servings,0,PT30M,#잡채만들기 #간단하게 만드는 잡채 #한꺼번에 볶아내면 금방 뚝딱 만드는 잡채!!!...
2,춤추는곰,2019-03-21,푹 익힌 닭에 깊은 양념맛이 좋은 닭볶음탕,5,6908917,"닭볶음용 닭 1마리,양파 1/2개,당근 1/3개,감자 1개,대파 1줌,고추장 3T,...",[요즘 마트에가면 닭볶음용 온가족세트 닭이 있더라구요~ 할인도 자주해서 종종 사오는...,3 servings,7,PT30M,"푹 익힌 닭에 깊은 양념맛이 좋은 닭볶음탕닭볶음용 닭 1마리,양파 1/2개,당근 1..."
3,왕눈이2,2019-09-15,맵지않아 더욱더 맛나다는 백김치의 비법은?,5,6919155,"배추 2포기,양파 1/2개,사과 1/2개,배 중간크기1개,당근 1/2개,무우 1/2...",[배추 2포기를 4등분으로 잘라주신후 큰볼에 물을 담아 소금 1컵을 풀어서 자른배추...,6 servings,3,PT2H,"맵지않아 더욱더 맛나다는 백김치의 비법은?배추 2포기,양파 1/2개,사과 1/2개,..."
4,EMYY,2019-07-01,메밀부꾸미,0,6915060,"메밀가루 3T,계란 1개,우유 3T,3층 치즈 2장,아몬드슬라이스 조금,건크랜베리 ...","[껍질벗긴 메밀3T를 분쇄기로 갈아요, (시판 메밀가루 3T 쓰셔도 되구요, )\...",1 servings,0,PT15M,"메밀부꾸미메밀가루 3T,계란 1개,우유 3T,3층 치즈 2장,아몬드슬라이스 조금,건..."


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def get_cosine_similarity(df: pd.DataFrame, column: str) -> np.ndarray:
    """
    특정 열에 대한 코사인 유사도를 반환하는 메소드
    """

    tokenized_data = df[column].fillna('')
    vectorizer = TfidfVectorizer()
    array = vectorizer.fit_transform(tokenized_data).todense()
    return cosine_similarity(array, array)

In [37]:
def make_similar_index(df: pd.DataFrame) -> np.ndarray:
    """
    코사인 유사도 합을 반환하는 메소드
    """

    similarity = get_cosine_similarity(df, "mergeIngredient")
    return similarity.argsort()[:, ::-1]

SIMILAR_INDEX = make_similar_index(data)
SIMILAR_INDEX

array([[   0, 1296,  546, ..., 3852, 3846, 3131],
       [   1, 1476, 1527, ..., 5342, 6325, 6826],
       [   2, 7662, 2237, ..., 3224, 7622, 3392],
       ...,
       [8205, 6241, 7625, ..., 5057, 5059, 4103],
       [8206,  997, 1562, ..., 2082, 2079, 8207],
       [8207, 1782, 6504, ..., 3313, 5589, 3395]])

In [38]:
def get_similar_places(base_df: pd.DataFrame, target_df: pd.DataFrame, column: str, num_display=5) -> pd.DataFrame:
    """
    코사인 유사도에 기반하여 특정 조건을 만족하는 행과 유사한 데이터프레임을 반환하는 메소드
    """

    value = target_df.iloc[0][column]
    index = base_df[base_df[column] == value].index.values
    max_index = min(num_display*2, len(base_df))

    similar_df = base_df.loc[SIMILAR_INDEX[index,1:max_index][0]]
    target_df = target_df.append(similar_df)
    target_df.drop_duplicates([column], inplace=True)

    return target_df.iloc[:num_display] if len(target_df) > num_display else target_df

In [55]:
data.iloc[[15]]

,author,createDate,name,ratingValue,recipeId,recipeIngredient,recipeInstructions,recipeYield,reviewCount,totalTime,mergeIngredient
15,로운이의식단일기,2020-09-19,"아기어묵 만들기 (no 밀가루, no 전분가루)",5,6942297,"새우 6마리,오징어 2마리,명태포 6장,연근 5cm,달걀 1개,양파 1/4개,대파 ...","[재료를 준비해요, 새우(6마리), 오징어(작은사이즈 2마리), 명태포(6장), ...",6 servings,1,PT0H60M,"아기어묵 만들기 (no 밀가루, no 전분가루)새우 6마리,오징어 2마리,명태포 6..."


In [56]:
results = get_similar_places(data, data.iloc[[15]], "mergeIngredient")
results

/tmp/ipykernel_35/955746744.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  target_df = target_df.append(similar_df)


,author,createDate,name,ratingValue,recipeId,recipeIngredient,recipeInstructions,recipeYield,reviewCount,totalTime,mergeIngredient
15,로운이의식단일기,2020-09-19,"아기어묵 만들기 (no 밀가루, no 전분가루)",5,6942297,"새우 6마리,오징어 2마리,명태포 6장,연근 5cm,달걀 1개,양파 1/4개,대파 ...","[재료를 준비해요, 새우(6마리), 오징어(작은사이즈 2마리), 명태포(6장), ...",6 servings,1,PT0H60M,"아기어묵 만들기 (no 밀가루, no 전분가루)새우 6마리,오징어 2마리,명태포 6..."
6667,yejib1112,2016-09-28,간단하게 끓여 호로록 호로록 애호박계란국,5,6857624,"애호박 1/2개,달걀 2개,간장 1/2T,소금 1/2t,파 약간,후추 약간,다시마 ...","[분량의 재료를 준비해주세요, 달걀은 소금 한꼬집 넣어 잘 풀고, 애호박은 채 썰...",3 servings,124,PT30M,"간단하게 끓여 호로록 호로록 애호박계란국애호박 1/2개,달걀 2개,간장 1/2T,소..."
8095,pipit****,2017-07-15,꿀처 육회,5,6872781,"소고기 300g,배 작은거 1개,계란 1개,파 5cm,마늘 4쪽,잣 3알,소금 0....","[고기는 한우 우둔살로 300g(반근) 준비해주시고 나머지 부재료인 배와 파, 마늘...",2 servings,6,PT30M,"꿀처 육회소고기 300g,배 작은거 1개,계란 1개,파 5cm,마늘 4쪽,잣 3알,..."
6675,yejib1112,2018-04-03,뜨끈한 국물이 땡길 때는! 표고버섯 어묵국,4,6886624,"물 약 600ml,무 1/4토막,양파 작은 거 1/2개,표고버섯 3-4개,어묵 2장...","[냄비에 물과 멸치, 다시마를 넣어 센불에서 끓이다 끓어오르면 중약불로 줄여 약 1...",4 servings,1,PT30M,"뜨끈한 국물이 땡길 때는! 표고버섯 어묵국물 약 600ml,무 1/4토막,양파 작은..."
7086,장네오,2019-02-15,삼겹살로 푸짐하게~ 제육김치볶음,5,6906683,"삼겹살 400g,배추김치 먹고싶은만큼,두부 1/2모,양파 1/2개,당근 얇게썰은것 ...",[달궈진 오목한 팬에 삼겹살을 넣고 중불로 볶아요\n기름이 나오기 시작하면 김치도 ...,2 servings,4,PT30M,"삼겹살로 푸짐하게~ 제육김치볶음삼겹살 400g,배추김치 먹고싶은만큼,두부 1/2모,..."
